In [41]:
from PIL import Image, ImageFilter
import numpy as np
from constants.behavioral_constants import *
from svgtrace import trace

letters_to_features = {
    "L": "ESCHER",
    "P": "POLKADOT",
    "R": "RIPPLE",
    "S": "SWIRL",
    "C": "CIRCLE",
    "Q": "SQUARE",
    "S": "STAR",
    "T": "TRIANGLE",
}


### Patterns

In [13]:
# Open the uploaded image
patterns = ["L", "P", "R", "S"]
for pattern in patterns: 
    image_path = f"/data/patrick_res/figures/wcst_paper/card_bitmaps/QB{pattern}.bmp"
    img = Image.open(image_path).convert("RGBA")

    # Convert image to numpy array
    data = np.array(img)

    # Define the cyan color range to replace (tolerance for similar shades)
    cyan_rgb = np.array([0, 255, 255])  # pure cyan
    tolerance = 150  # adjust if needed

    # Create a mask for pixels close to cyan

    cyan_mask = np.all(np.abs(data[:, :, :3] - cyan_rgb) < tolerance, axis=-1)
    grey_mask = np.all(np.abs(data[:, :, :3] - np.array([100,100,100])) < 2, axis=-1)
    # Replace cyan pixels with white
    data[cyan_mask + grey_mask] = [255, 255, 255, 255]  # white

    # Convert back to image
    result_img = Image.fromarray(data)

    # Save processed image
    output_path = f"/data/patrick_res/figures/wcst_paper/feature_bitmaps/{letters_to_features[pattern]}.bmp"
    result_img.save(output_path)
    output_path

### Colors: 

In [6]:
colors_rgb = {
    "CYAN": np.array([0, 255, 255, 255]),
    "YELLOW": np.array([255,255,0, 255]),
    "MAGENTA": np.array([255,0,255, 255]),
    "GREEN": np.array([33,221,0, 255]),
}
image_path = "/data/patrick_res/figures/wcst_paper/card_bitmaps/QBL.bmp"
example_image = np.array(Image.open(image_path).convert("RGBA"))

In [7]:
for color_name in colors_rgb:
    rgb = colors_rgb[color_name]
    color_square = np.full(example_image.shape, rgb, dtype=np.uint8)
    color_img = Image.fromarray(color_square)
    output_path = f"/data/patrick_res/figures/wcst_paper/feature_bitmaps/{color_name}.bmp"
    color_img.save(output_path)

### Shapes:

In [21]:
# Open the uploaded image
shapes = ["C", "Q", "S", "T"]
for shape in shapes: 
    image_path = f"/data/patrick_res/figures/wcst_paper/card_bitmaps/{shape}BP.bmp"
    img = Image.open(image_path).convert("RGBA")

    # Convert image to numpy array
    data = np.array(img)

    not_grey_mask = np.any(np.abs(data[:, :, :3] - np.array([100,100,100])) != 0, axis=-1)
    # Replace cyan pixels with white
    data[not_grey_mask] = [255, 255, 255, 255]  # white

    # Convert back to image
    result_img = Image.fromarray(data)

    # Save processed image
    output_path = f"/data/patrick_res/figures/wcst_paper/feature_bitmaps/{letters_to_features[shape]}_grey.bmp"
    result_img.save(output_path)
    output_path

In [40]:
edge_threshold = 30

for pattern in POSSIBLE_FEATURES["Shape"]:
    img = Image.open(f"/data/patrick_res/figures/wcst_paper/feature_bitmaps/{pattern}_grey.bmp").convert("L")
    outline_img = img.filter(ImageFilter.FIND_EDGES)
    outline_img = outline_img.filter(ImageFilter.GaussianBlur(radius=1))
    width, height = outline_img.size
    # Create a new image with a pure white background for the final output
    final_output_img = Image.new("RGB", (width, height), (255, 255, 255))
    final_pixels = final_output_img.load() # Get pixel access for the final image

    # Load pixels from the edge-detected image for processing
    outline_pixels = outline_img.load()

    # Iterate through the edge-detected image and apply a threshold.
    # If a pixel is dark enough (below the threshold), it's part of the edge,
    # so we color it black in our final output. Otherwise, it remains white.
    for y in range(height):
        for x in range(width):
            # Check the grayscale value of the pixel in the edge-detected image
            if outline_pixels[x, y] > edge_threshold:
                final_pixels[x, y] = (0, 0, 0) # Set to black for the outline

    final_output_img.save(f"/data/patrick_res/figures/wcst_paper/feature_bitmaps/{pattern}.bmp")